In [31]:
import os
import numpy as np
from datetime import datetime
path = os.getcwd()
if path[1:5] == "Home":
    %run /Home/siv30/wad005/master/GRB/src/harvest_matlab_data.ipynb
    %run /Home/siv30/wad005/master/GRB/src/convert_erg.ipynb
    %run /Home/siv30/wad005/master/GRB/src/safety_time.ipynb
else:
    %run /Users/andreas/phys/master/GRB/src/harvest_matlab_data.ipynb
    %run /Users/andreas/phys/master/GRB/src/convert_erg.ipynb
    %run /Users/andreas/phys/master/GRB/src/safety_time.ipynb


All MAC paths are set!

All MAC paths are set!
read_matlab_file.ipynb loaded successfully!

All MAC paths are set!
read_matlab_file.ipynb loaded successfully!
convert_erg.ipynb loaded successfully!

All MAC paths are set!
read_matlab_file.ipynb loaded successfully!

All MAC paths are set!
read_matlab_file.ipynb loaded successfully!
convert_erg.ipynb loaded successfully!
safety_time.ipynb loaded successfully!


In [59]:
def get_count_rate(fn=None, T0=None, bkg_end= -2.0):
    '''
    fn <str> name of *.mat
    T0 <str> trigger time in iso format 
    bkg_end <float> where bkg spectrum end in [s] assuming the bkg is before burst
    '''
    
    T0 = datetime.fromisoformat(T0)
    print(T0)
    print('bkg ends at: ',bkg_end, 's')
    df = harvest_data(fn=fn,T0=T0.isoformat())
    i_bkg_end = find_nearest(df['tus'].to_numpy(),bkg_end)
    df_bkg = df.truncate(after=i_bkg_end-1)

    tus,erg,dau,det = df_bkg['tus'].to_numpy(),df_bkg['erg'].to_numpy(),df_bkg['dau'].to_numpy(),df_bkg['det'].to_numpy()
    keV,abc = convert_erg(erg,dau,det,T0)
    df_bkg['keV'] = keV
    df_bkg = df_bkg.sort_values(by=['adr','tus'],ignore_index=True)
    df_bkg['tus'] = df_bkg['tus']*1e6

    st,dt = get_st(df_bkg),get_dt(df_bkg)
    df_bkg['st']=st
    df_bkg['dt']=dt

    flag = get_flag(df_bkg)
    df_bkg['flag']=flag
    print('safety time: ',np.round((len(np.where(flag==True)[0])/len(flag))*100,3), '% removed')
    df_bkg.drop(labels=np.where(flag==True)[0],inplace=True)
    df_bkg = df_bkg.sort_values(by='tus')

    tus = df_bkg['tus'].to_numpy(dtype=np.float64)
    exposure = (tus[-1]-tus[0])/1e6 #seconds
    count_rate =len(tus)/exposure
    print('background count rate: ',np.round(count_rate,3), ' counts/s')
    return count_rate

In [60]:
cr = get_count_rate(fn='2019_3_5_13_3_48',T0='2019-03-05T13:05:19.816')

2019-03-05 13:05:19.816000
bkg ends at:  -2.0 s
dau/det starting from index 1
tus array given in seconds
df sorted by tus
safety time:  0.154 % removed
background count rate:  4194.389  counts/s
